In [ ]:
# Intend to predict answer upvotes(CNN)

In [28]:
from __future__ import print_function
import keras
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from keras import backend as K
import tensorflow as tf
import horovod.keras as hvd

# Horovod: initialize Horovod.
hvd.init()

# Horovod: pin GPU to be used to process local rank (one GPU per process)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(hvd.local_rank())
K.set_session(tf.Session(config=config))

batch_size = 1024
num_classes = 10
validation_ratio = 0.1

# Enough epochs to demonstrate learning rate warmup and the reduction of
# learning rate when training plateaues.
epochs = 24

def dataGenerator(validation = False):
    while True:
        questionOriginalFile = pd.read_csv("../Questions.csv",encoding="utf-8", iterator = True, chunksize=batch_size)
        current_chunk_index = 0
        for chunk in questionOriginalFile:
    #         print(chunk["Body"])
            current_chunk_index += 1
            if current_chunk_index % 100 < int(100 * validation_ratio) and not validation:
                continue
            if current_chunk_index % 100 >= int(100 * validation_ratio) and validation:
                continue
            chunk["Body"] = chunk["Body"].map(lambda x: one_hot(x, 10000))
#             chunk["Score"] = chunk["Score"].map(lambda x: 1 if x > 0 else 0)
            chunk["Score"] = chunk["Score"].map(lambda x: 1 if x > 100 else x / 100)
    #         print(chunk["Score"])
            body = pd.DataFrame(item for item in chunk["Body"])
            body = body.as_matrix()
            body = np.nan_to_num(body)
    #         chunk["Body"].astype("int32").as_matrix()
            score = chunk["Score"].as_matrix()
            body = pad_sequences(body, maxlen=1000, dtype='int32', padding='post', truncating='post', value=0.0)
            yield body, score
            

model = Sequential()
model.add(Embedding(10000, 100, input_length = 1000))
model.add(Conv1D(32, kernel_size=3,
                 activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Horovod: adjust learning rate based on number of GPUs.
opt = keras.optimizers.Adam()

# Horovod: adjust learning rate based on number of GPUs.
opt = keras.optimizers.Adam()

# Horovod: add Horovod Distributed Optimizer.
opt = hvd.DistributedOptimizer(opt)

model.compile(loss=keras.losses.mse,
              optimizer=opt,
              metrics=['mse', 'mae', 'mape'])
K.get_session().run(tf.global_variables_initializer())


callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),

    # Horovod: average metrics among workers at the end of every epoch.
    #
    # Note: This callback must be in the list before the ReduceLROnPlateau,
    # TensorBoard or other metrics-based callbacks.
    hvd.callbacks.MetricAverageCallback(),

    # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
    # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
    # the first five epochs. See https://arxiv.org/abs/1706.02677 for details.
#     hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),

    # Reduce the learning rate if training plateaues.
    keras.callbacks.ReduceLROnPlateau(patience=10, verbose=1),
]

# Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoint-{epoch:02d}-{val_mean_squared_error:.4f}.hdf5', \
                                                    monitor='val_mean_squared_error', verbose=1, save_best_only=True, mode='min'))

# print(next(dataGenerator()))
# Train the model.
# Horovod: the training will randomly sample 1 / N batches of training data and
# 3 / N batches of validation data on every worker, where N is the number of workers.
# Over-sampling of validation data helps to increase probability that every validation
# example will be evaluated.
model.fit_generator(dataGenerator(validation = False),
                    steps_per_epoch = 100 // hvd.size(),
                    callbacks=callbacks,
                    epochs=epochs,
                    verbose=1,
                    validation_data=dataGenerator(validation = True),
                    validation_steps = 10 // hvd.size())

Epoch 1/24
100/100 [==============================] - 154s 2s/step - loss: 0.0184 - mean_squared_error: 0.0184 - mean_absolute_error: 0.0523 - mean_absolute_percentage_error: 1639622.0252 - val_loss: 0.0313 - val_mean_squared_error: 0.0313 - val_mean_absolute_error: 0.0719 - val_mean_absolute_percentage_error: 77.6172

Epoch 00001: val_mean_squared_error improved from inf to 0.03130, saving model to ./checkpoint-01-0.0313.hdf5
Epoch 2/24
100/100 [==============================] - 50s 501ms/step - loss: 0.0073 - mean_squared_error: 0.0073 - mean_absolute_error: 0.0298 - mean_absolute_percentage_error: 64.3789 - val_loss: 0.0205 - val_mean_squared_error: 0.0205 - val_mean_absolute_error: 0.0539 - val_mean_absolute_percentage_error: 72.3926

Epoch 00002: val_mean_squared_error improved from 0.03130 to 0.02052, saving model to ./checkpoint-02-0.0205.hdf5
Epoch 3/24
100/100 [==============================] - 44s 436ms/step - loss: 0.0048 - mean_squared_error: 0.0048 - mean_absolute_error: 0


Epoch 00020: val_mean_squared_error did not improve
Epoch 21/24
100/100 [==============================] - 101s 1s/step - loss: 4.4854e-04 - mean_squared_error: 4.4854e-04 - mean_absolute_error: 0.0084 - mean_absolute_percentage_error: 43.7394 - val_loss: 0.0059 - val_mean_squared_error: 0.0059 - val_mean_absolute_error: 0.0249 - val_mean_absolute_percentage_error: 60.1465

Epoch 00021: val_mean_squared_error did not improve
Epoch 22/24
100/100 [==============================] - 96s 962ms/step - loss: 3.4717e-04 - mean_squared_error: 3.4717e-04 - mean_absolute_error: 0.0071 - mean_absolute_percentage_error: 39.0628 - val_loss: 0.0055 - val_mean_squared_error: 0.0055 - val_mean_absolute_error: 0.0242 - val_mean_absolute_percentage_error: 60.4980

Epoch 00022: val_mean_squared_error did not improve
Epoch 23/24
100/100 [==============================] - 70s 701ms/step - loss: 0.0150 - mean_squared_error: 0.0150 - mean_absolute_error: 0.0434 - mean_absolute_percentage_error: 67.9237 - val

In [ ]:
# Intend to predict answer upvotes(LSTM)

In [34]:
from __future__ import print_function
import keras
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from keras import backend as K
import tensorflow as tf
import horovod.keras as hvd

# Horovod: initialize Horovod.
hvd.init()

# Horovod: pin GPU to be used to process local rank (one GPU per process)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(hvd.local_rank())
K.set_session(tf.Session(config=config))

batch_size = 1024
num_classes = 10
validation_ratio = 0.1

# Enough epochs to demonstrate learning rate warmup and the reduction of
# learning rate when training plateaues.
epochs = 24

def dataGenerator(validation = False):
    while True:
        questionOriginalFile = pd.read_csv("../Questions.csv",encoding="utf-8", iterator = True, chunksize=batch_size)
        current_chunk_index = 0
        for chunk in questionOriginalFile:
    #         print(chunk["Body"])
            current_chunk_index += 1
            if current_chunk_index % 100 < int(100 * validation_ratio) and not validation:
                continue
            if current_chunk_index % 100 >= int(100 * validation_ratio) and validation:
                continue
            chunk["Body"] = chunk["Body"].map(lambda x: one_hot(x, 10000))
#             chunk["Score"] = chunk["Score"].map(lambda x: 1 if x > 0 else 0)
            chunk["Score"] = chunk["Score"].map(lambda x: 1 if x > 100 else x / 100)
    #         print(chunk["Score"])
            body = pd.DataFrame(item for item in chunk["Body"])
            body = body.as_matrix()
            body = np.nan_to_num(body)
    #         chunk["Body"].astype("int32").as_matrix()
            score = chunk["Score"].as_matrix()
            body = pad_sequences(body, maxlen=1000, dtype='int32', padding='post', truncating='post', value=0.0)
            yield body, score
            

model = Sequential()
model.add(Embedding(10000, 100, input_length = 1000))
model.add(LSTM(100, return_sequences = True))
# model.add(Conv1D(64, 3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.25))
model.add(LSTM(100, return_sequences = False))
# model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Horovod: adjust learning rate based on number of GPUs.
opt = keras.optimizers.Adam()

# Horovod: add Horovod Distributed Optimizer.
opt = hvd.DistributedOptimizer(opt)

model.compile(loss=keras.losses.mse,
              optimizer=opt,
              metrics=['mse', 'mae', 'mape'])
K.get_session().run(tf.global_variables_initializer())


callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),

    # Horovod: average metrics among workers at the end of every epoch.
    #
    # Note: This callback must be in the list before the ReduceLROnPlateau,
    # TensorBoard or other metrics-based callbacks.
    hvd.callbacks.MetricAverageCallback(),

    # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
    # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
    # the first five epochs. See https://arxiv.org/abs/1706.02677 for details.
    hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),

    # Reduce the learning rate if training plateaues.
    keras.callbacks.ReduceLROnPlateau(patience=10, verbose=1),
]

# Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoint-{epoch:02d}-{val_mean_squared_error:.4f}.hdf5', \
                                                    monitor='val_mean_squared_error', verbose=1, save_best_only=True, mode='min'))

# print(next(dataGenerator()))
# Train the model.
# Horovod: the training will randomly sample 1 / N batches of training data and
# 3 / N batches of validation data on every worker, where N is the number of workers.
# Over-sampling of validation data helps to increase probability that every validation
# example will be evaluated.
model.fit_generator(dataGenerator(validation = False),
                    steps_per_epoch = 100 // hvd.size(),
                    callbacks=callbacks,
                    epochs=epochs,
                    verbose=1,
                    validation_data=dataGenerator(validation = True),
                    validation_steps = 10 // hvd.size())

# # Evaluate the model on the full data set.
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Epoch 1/24
100/100 [==============================] - 345s 3s/step - loss: 0.0282 - mean_squared_error: 0.0282 - mean_absolute_error: 0.0791 - mean_absolute_percentage_error: 13993677.3475 - val_loss: 0.0284 - val_mean_squared_error: 0.0284 - val_mean_absolute_error: 0.0658 - val_mean_absolute_percentage_error: 5595068.6000

Epoch 00001: val_mean_squared_error improved from inf to 0.02835, saving model to ./checkpoint-01-0.0284.hdf5
Epoch 2/24
100/100 [==============================] - 309s 3s/step - loss: 0.0068 - mean_squared_error: 0.0068 - mean_absolute_error: 0.0326 - mean_absolute_percentage_error: 7466312.9800 - val_loss: 0.0159 - val_mean_squared_error: 0.0159 - val_mean_absolute_error: 0.0442 - val_mean_absolute_percentage_error: 4025208.4000

Epoch 00002: val_mean_squared_error improved from 0.02835 to 0.01592, saving model to ./checkpoint-02-0.0159.hdf5
Epoch 3/24
100/100 [==============================] - 306s 3s/step - loss: 0.0045 - mean_squared_error: 0.0045 - mean_absol

100/100 [==============================] - 305s 3s/step - loss: 7.6944e-04 - mean_squared_error: 7.6944e-04 - mean_absolute_error: 0.0117 - mean_absolute_percentage_error: 2022775.3462 - val_loss: 0.0034 - val_mean_squared_error: 0.0034 - val_mean_absolute_error: 0.0207 - val_mean_absolute_percentage_error: 713647.3500

Epoch 00020: val_mean_squared_error improved from 0.00369 to 0.00336, saving model to ./checkpoint-20-0.0034.hdf5
Epoch 21/24
100/100 [==============================] - 305s 3s/step - loss: 4.3907e-04 - mean_squared_error: 4.3907e-04 - mean_absolute_error: 0.0092 - mean_absolute_percentage_error: 1479617.6838 - val_loss: 0.0030 - val_mean_squared_error: 0.0030 - val_mean_absolute_error: 0.0197 - val_mean_absolute_percentage_error: 412209.8719

Epoch 00021: val_mean_squared_error improved from 0.00336 to 0.00299, saving model to ./checkpoint-21-0.0030.hdf5
Epoch 22/24
100/100 [==============================] - 309s 3s/step - loss: 3.4396e-04 - mean_squared_error: 3.4396e

In [14]:
score = model.evaluate_generator(dataGenerator(validation = True), 100)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.008448001376236788
Test accuracy: 0.008448001376236788


In [15]:
print(score)

[0.008448001376236788, 0.008448001376236788, 0.034822447281330826, 10765704.1175]


In [29]:
predictData = next(dataGenerator(validation = True))
print(predictData)

(array([[9733, 6599, 6705, ...,    0,    0,    0],
       [9733,  829, 3821, ...,    0,    0,    0],
       [9733, 3410, 1751, ...,    0,    0,    0],
       ...,
       [9733, 5488, 1078, ...,    0,    0,    0],
       [9733, 7434, 5551, ...,    0,    0,    0],
       [9733, 6562, 5488, ...,    0,    0,    0]], dtype=int32), array([0.26, 1.  , 0.21, ..., 0.05, 0.01, 1.  ]))


In [30]:
result = model.predict(predictData[0])

In [31]:
predictData[1]

array([0.26, 1.  , 0.21, ..., 0.05, 0.01, 1.  ])

In [32]:
for index in range(len(predictData[1])):
    print(predictData[1][index], result[index])

0.26 [4.589336e-22]
1.0 [3.9083568e-23]
0.21 [5.872131e-23]
0.53 [4.3699442e-23]
0.49 [2.5148252e-22]
0.29 [2.0163759e-22]
0.13 [4.296979e-23]
0.21 [4.737998e-22]
0.79 [1.1739499e-22]
0.09 [2.623257e-22]
0.28 [3.139135e-23]
0.14 [7.8038904e-23]
0.42 [4.7563586e-23]
0.17 [3.6593683e-23]
0.36 [5.938789e-23]
0.17 [6.0162944e-23]
0.23 [9.624909e-23]
0.18 [6.218364e-23]
0.18 [3.713367e-23]
0.63 [7.181113e-23]
0.51 [6.0691266e-23]
0.13 [3.7145002e-23]
0.1 [3.1323647e-23]
0.77 [7.191393e-23]
0.83 [4.900046e-23]
0.1 [2.8790504e-23]
1.0 [3.294746e-23]
0.39 [3.444852e-23]
0.16 [4.3435357e-23]
0.14 [3.7650586e-23]
0.07 [5.8845546e-23]
0.27 [4.3496042e-23]
0.12 [1.04315325e-22]
0.32 [5.1579976e-23]
0.14 [2.6617129e-22]
0.17 [2.515754e-16]
0.76 [4.0114937e-23]
0.29 [5.1434378e-23]
0.25 [4.1778862e-23]
0.46 [4.584139e-22]
0.25 [3.0358197e-22]
0.12 [1.3592468e-22]
0.16 [2.879907e-23]
0.41 [4.765439e-23]
0.03 [2.2419513e-22]
0.92 [8.967194e-23]
0.07 [5.052602e-23]
0.09 [4.8220536e-23]
0.34 [9.5438166e

0.08 [4.869765e-23]
0.69 [4.2752983e-23]
0.11 [3.3180676e-23]
0.75 [3.271277e-23]
0.33 [1.4544646e-22]
0.03 [3.5127288e-23]
0.0 [1.127139e-22]
0.02 [1.164321e-22]
0.13 [3.8308177e-23]
0.0 [3.71925e-23]
0.05 [8.788323e-22]
0.01 [6.9960293e-23]
0.02 [6.366144e-23]
0.04 [6.99667e-23]
0.05 [6.800912e-23]
0.15 [3.00627e-23]
0.42 [1.1047816e-22]
1.0 [1.0685714e-22]
0.02 [3.104564e-23]
0.04 [7.412175e-23]
0.02 [4.1463567e-22]
0.06 [3.253853e-23]
0.78 [4.246901e-23]
0.37 [2.2101505e-22]
0.09 [3.580577e-23]
0.02 [4.5078358e-23]
0.04 [7.13415e-23]
1.0 [3.6005193e-23]
0.05 [9.499213e-23]
0.03 [3.6244284e-23]
0.05 [2.9579917e-23]
0.05 [6.2866496e-23]
0.05 [6.773262e-23]
0.91 [2.6190376e-22]
0.1 [3.7485514e-22]
0.06 [2.9574613e-23]
0.08 [4.5878102e-23]
0.14 [4.6432006e-23]
0.04 [3.798093e-23]
0.1 [3.1469886e-23]
0.0 [4.6580498e-23]
0.12 [1.079593e-22]
0.33 [2.1243147e-21]
0.09 [5.6695137e-23]
0.02 [5.176309e-23]
1.0 [1.9355456e-22]
0.06 [1.7624888e-22]
0.1 [3.3096244e-22]
0.31 [7.2062527e-22]
0.0 [

In [33]:
result

array([[4.5893358e-22],
       [3.9083568e-23],
       [5.8721310e-23],
       ...,
       [3.2262185e-22],
       [1.2876652e-22],
       [7.7611399e-23]], dtype=float32)

In [35]:
result = model.predict(predictData[0])
for index in range(len(predictData[1])):
    print(predictData[1][index], result[index])

0.26 [0.0155145]
1.0 [0.0155145]
0.21 [0.0155145]
0.53 [0.0155145]
0.49 [0.0155145]
0.29 [0.0155145]
0.13 [0.0155145]
0.21 [0.0155145]
0.79 [0.0155145]
0.09 [0.0155145]
0.28 [0.0155145]
0.14 [0.0155145]
0.42 [0.0155145]
0.17 [0.0155145]
0.36 [0.0155145]
0.17 [0.0155145]
0.23 [0.0155145]
0.18 [0.0155145]
0.18 [0.0155145]
0.63 [0.0155145]
0.51 [0.0155145]
0.13 [0.0155145]
0.1 [0.0155145]
0.77 [0.0155145]
0.83 [0.0155145]
0.1 [0.0155145]
1.0 [0.0155145]
0.39 [0.0155145]
0.16 [0.0155145]
0.14 [0.0155145]
0.07 [0.0155145]
0.27 [0.0155145]
0.12 [0.0155145]
0.32 [0.0155145]
0.14 [0.0155145]
0.17 [0.01518357]
0.76 [0.0155145]
0.29 [0.0155145]
0.25 [0.0155145]
0.46 [0.0155145]
0.25 [0.0155145]
0.12 [0.0155145]
0.16 [0.0155145]
0.41 [0.0155145]
0.03 [0.0155145]
0.92 [0.0155145]
0.07 [0.0155145]
0.09 [0.0155145]
0.34 [0.0155145]
0.02 [0.0155145]
0.46 [0.0155145]
0.11 [0.0155145]
0.08 [0.0155145]
0.15 [0.0155145]
0.16 [0.0155145]
0.08 [0.0155145]
0.11 [0.0155145]
0.22 [0.0155145]
0.11 [0.0155145]


In [ ]:
# Intend to predict whether the answer score is greater than 0 or not(LSTM)

In [44]:
from __future__ import print_function
import keras
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from keras import backend as K
import tensorflow as tf
import horovod.keras as hvd

# Horovod: initialize Horovod.
hvd.init()

# Horovod: pin GPU to be used to process local rank (one GPU per process)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(hvd.local_rank())
K.set_session(tf.Session(config=config))

batch_size = 1024
num_classes = 10
validation_ratio = 0.1

# Enough epochs to demonstrate learning rate warmup and the reduction of
# learning rate when training plateaues.
epochs = 24

def dataGenerator(validation = False):
    while True:
        questionOriginalFile = pd.read_csv("../Questions.csv",encoding="utf-8", iterator = True, chunksize=batch_size)
        current_chunk_index = 0
        for chunk in questionOriginalFile:
    #         print(chunk["Body"])
            current_chunk_index += 1
            if current_chunk_index % 100 < int(100 * validation_ratio) and not validation:
                continue
            if current_chunk_index % 100 >= int(100 * validation_ratio) and validation:
                continue
            chunk["Body"] = chunk["Body"].map(lambda x: one_hot(x, 10000))
#             chunk["Score"] = chunk["Score"].map(lambda x: 1 if x > 0 else 0)
            chunk["Score"] = chunk["Score"].map(lambda x: 1 if x >= 10 else 0)
    #         print(chunk["Score"])
            body = pd.DataFrame(item for item in chunk["Body"])
            body = body.as_matrix()
            body = np.nan_to_num(body)
    #         chunk["Body"].astype("int32").as_matrix()
            score = chunk["Score"].as_matrix()
            body = pad_sequences(body, maxlen=1000, dtype='int32', padding='post', truncating='post', value=0.0)
            yield body, score
            

model = Sequential()
model.add(Embedding(10000, 100, input_length = 1000))
model.add(LSTM(100, return_sequences = True))
# model.add(Conv1D(64, 3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.25))
model.add(LSTM(100, return_sequences = False))
# model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Horovod: adjust learning rate based on number of GPUs.
opt = keras.optimizers.Adam()

# Horovod: add Horovod Distributed Optimizer.
opt = hvd.DistributedOptimizer(opt)

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])
K.get_session().run(tf.global_variables_initializer())


callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),

    # Horovod: average metrics among workers at the end of every epoch.
    #
    # Note: This callback must be in the list before the ReduceLROnPlateau,
    # TensorBoard or other metrics-based callbacks.
    hvd.callbacks.MetricAverageCallback(),

    # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
    # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
    # the first five epochs. See https://arxiv.org/abs/1706.02677 for details.
#     hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),

    # Reduce the learning rate if training plateaues.
    keras.callbacks.ReduceLROnPlateau(patience=10, verbose=1),
]

# Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoint-{epoch:02d}-{val_acc:.4f}.hdf5', \
                                                    monitor='val_acc', verbose=1, save_best_only=True, mode='max'))

# print(next(dataGenerator()))
# Train the model.
# Horovod: the training will randomly sample 1 / N batches of training data and
# 3 / N batches of validation data on every worker, where N is the number of workers.
# Over-sampling of validation data helps to increase probability that every validation
# example will be evaluated.
model.fit_generator(dataGenerator(validation = False),
                    steps_per_epoch = 20 // hvd.size(),
                    callbacks=callbacks,
                    epochs=epochs,
                    verbose=1,
                    validation_data=dataGenerator(validation = True),
                    validation_steps = 2 // hvd.size())

# # Evaluate the model on the full data set.
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Epoch 1/24
20/20 [==============================] - 67s 3s/step - loss: 0.4804 - acc: 0.8562 - val_loss: 0.8480 - val_acc: 0.6274

Epoch 00001: val_acc improved from -inf to 0.62744, saving model to ./checkpoint-01-0.6274.hdf5
Epoch 2/24
20/20 [==============================] - 65s 3s/step - loss: 0.3630 - acc: 0.8860 - val_loss: 0.4011 - val_acc: 0.8618

Epoch 00002: val_acc improved from 0.62744 to 0.86182, saving model to ./checkpoint-02-0.8618.hdf5
Epoch 3/24
20/20 [==============================] - 65s 3s/step - loss: 0.3266 - acc: 0.9027 - val_loss: 0.2837 - val_acc: 0.9185

Epoch 00003: val_acc improved from 0.86182 to 0.91846, saving model to ./checkpoint-03-0.9185.hdf5
Epoch 4/24
20/20 [==============================] - 65s 3s/step - loss: 0.2929 - acc: 0.9173 - val_loss: 0.2657 - val_acc: 0.9253

Epoch 00004: val_acc improved from 0.91846 to 0.92529, saving model to ./checkpoint-04-0.9253.hdf5
Epoch 5/24
20/20 [==============================] - 66s 3s/step - loss: 0.2906 - ac

In [47]:
iterator = dataGenerator(validation = True)

In [ ]:
predictData = next(iterator)
result = model.predict(predictData[0])
for index in range(len(predictData[1])):
    print(predictData[1][index], result[index])

0 [0.03486204]
1 [0.02626682]
0 [0.02626681]
1 [0.02626681]
0 [0.02626682]
1 [0.02626681]
0 [0.02626682]
0 [0.02626682]
0 [0.02626681]
0 [0.02626681]
0 [0.02626681]
0 [0.02626681]
0 [0.02626682]
0 [0.02626681]
1 [0.02626681]
0 [0.02626681]
1 [0.02626681]
0 [0.02626682]
0 [0.02626681]
0 [0.02626681]
1 [0.02626682]
0 [0.02626681]
0 [0.02626681]
0 [0.02626681]
0 [0.02626682]
1 [0.02626682]
0 [0.02626682]
0 [0.02626681]
1 [0.02626681]
1 [0.02626682]
1 [0.02626681]
1 [0.02626682]
0 [0.02626681]
1 [0.02626682]
1 [0.02626681]
0 [0.02626681]
0 [0.02626681]
1 [0.02626682]
0 [0.02626681]
0 [0.02626682]
0 [0.02626681]
0 [0.02626682]
0 [0.02626681]
0 [0.02626681]
1 [0.02626682]
0 [0.02626681]
1 [0.02626682]
0 [0.02626682]
0 [0.02626681]
0 [0.02626681]
0 [0.02626682]
1 [0.02626681]
1 [0.02626681]
0 [0.02626681]
1 [0.02626682]
0 [0.02626681]
1 [0.02626681]
0 [0.02626681]
0 [0.02626681]
1 [0.02626681]
0 [0.02626682]
0 [0.02626681]
1 [0.02626682]
0 [0.02626681]
0 [0.02626681]
1 [0.02626681]
0 [0.02626

In [ ]:
scores = model.evaluate(predictData[0], predictData[1], verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))